# 計算機誤差

浮動小数点数は仮数部の桁数が限られているため，演算を行うたびに微小な誤差が発生する．

浮動小数点数演算から生じる誤差はしばしば**計算機誤差**と総称される．

## 丸め誤差
浮動小数点数どうしを四則演算して得られた値は浮動小数点数になるとは限らない．

In [4]:
1.111111111^2

1.2345679009876545

正確な値は $1.111111111^2 = 1.234567900987654321$ であり，この場合は下4桁分が切り捨てられている．

このように演算結果の値に近い浮動小数点数への近似を **丸め** (rounding) といい，
その際に生じる誤差を **丸め誤差** (rounding error)  という．

丸め誤差は浮動小数点数の計算において，ほぼ必然的に発生する．
丸め方には，正/負の無限大の方向，原点方向，最近点への丸めなどがある．

## 情報落ち
`1.0` と，その右隣にある浮動小数点数との距離を machine epsilon という．

倍精度の場合，先頭の桁を除いた仮数部が52bitなのでmachine epsilon は $0.00 \cdots 01_2 = 2^{-52}$ である．

Juliaでは，倍精度のmachine epsilonは `eps(1.0)` `eps(Float64)` や などで取得できる．

In [5]:
ϵ = eps(1.0)

2.220446049250313e-16

machine epsilon より小さな値を加算したとき，丸めによって値が変化しないことがある．

In [6]:
@show 1.0 + eps(1.0) 
@show 1.0 + 1e-16

1.0 + eps(1.0) = 1.0000000000000002
1.0 + 1.0e-16 = 1.0


1.0

![machine-epsilon](machine-epsilon.svg)

同様に，浮動小数点数 `x` の右隣りの浮動小数点数は `x + eps(x)` であるから，
`x` に `eps(x)`より小さな値を加算しても，丸めによって値が変化しないことがある．

これを**情報落ち**という．


`1.0 + 1e-16` の計算で情報落ちすることは避けることはできない．

しかし，計算順序を変えることで情報落ちを回避できる場合もある．

In [7]:
(1e-16 + 1.0) - 1.0

0.0

In [8]:
1e-16 + (1.0 - 1.0)

1.0e-16

##  桁落ち
$b$ を正の実数とする．
2次方程式　$f(x) = x^2 - 2bx + 1 = 0$ の解は，解の公式より
$$
  \alpha := b - \sqrt{b^2 - 1}, \quad \beta := b + \sqrt{b^2 - 1} 
$$
と表される．
$b^2 \gg 1$の場合，
$\alpha$をこのまま計算すると， $\sqrt{b^2 - 1} \approx b$ であるから仮数部が打ち消し合って有効桁数が減少する．

In [9]:
b = 1e8  # 1e8 = 10^8 
alpha = b - sqrt(b^2 - 1)

0.0

計算値は `0.0` であるが，これが解ではないことは代入してみれば明らかである．

このように値が近い２つの浮動小数点数の減算により有効桁数が減少することを**桁落ち**と呼ぶ．

これも計算の仕方を工夫することで回避できることがある．

## 桁落ちの回避



次のように式変形してから計算すると，真値に近い値が得られる．
$$
b - \sqrt{b^2 - 1}  = \frac{1}{b + \sqrt{b^2 - 1}} 
$$


In [10]:
b = 1e8
x = 1 / (b + sqrt(b^2 - 1))

5.0e-9

<div class="alert alert-warning">
Warning:  上の計算式は万能ではなく，b<0 かつ |b|が極めて大きい場合，桁落ちが発生する．
</div>

    